# Atomic C++ Data Types

Recall that C++ requires users to specify the **data type** of each variable. The **primary built-in** data types in C++ are: `int`, `float`, `double`, `bool`, and `char`. There is also a special type called `pointer` which holds a memory location. C++ also has a collection of [compound data types](https://www.learncpp.com/cpp-tutorial/introduction-to-compound-data-types/).

### Numeric data

These are `int`, `float`, and `double`. Note that there is no distinction between integer and float division in C++ (unlike in Python). That is, when two integers are divided, then integer division is done. Otherwise, it's float division whenever one of the numbers is a float.

Exponentiation is done using `pow()` from the `cmath` library and the remainder (modulo) operator is done with `%`. Modulo outputs `int` while `pow()` outputs `double`.

In [1]:
from utils import runcpp

In [2]:
%%runcpp
#include <iostream>
#include <cmath>
using namespace std;

int main(){
    auto print_type = [](auto expr) {
        cout << "(" << typeid(expr).name() << ") " << expr << endl;
    };

    // order of operations
    print_type(2 + 3 * 4);
    print_type((2 + 3) * 4);
    print_type((2 + 3.) * 4);

    // float & integer division
    cout << endl;
    print_type(7 / 3);
    print_type(float(7) / 3);
    print_type(7. / 3);
    print_type(6. / 3);

    // modulo & pow
    cout << endl;
    print_type(7 % 3);
    print_type(pow(2, 1));
    print_type(pow(2, 10));
    print_type(pow(2, 100));

    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

(i) 14
(i) 20
(d) 20

(i) 2
(f) 2.33333
(d) 2.33333
(d) 2

(i) 1
(d) 2
(d) 1024
(d) 1.26765e+30


Observe that C++ defaults to `double` (e.g. `7.` is of type `double`). The only result that is `float` is when we explicitly casted `float(7)`. Also, notice that the presence of a double in the operation promotes the `int` to double which makes sense.

Finally, we use a **lambda function** with automatically deduced input and output type `auto`. The `[]` is the **capture clause**. Empty `[]` means the lambda does not capture any variables from the surrounding scope.

**Remark.** Modifiers like `short`, `long`, and `unsigned` can optionally be used for numeric variables to help ensure that space is used as efficiently as possible.

## Boolean data

C++ uses the keyword `bool` which is not capitalized. The possible state values for a C++ Boolean are `true` and `false` (note: lower case). C++ uses the standard Boolean operators: `&&` (AND), `||` (OR), and `!` (NOT).

Note that the internally stored values representing `true` and `false` are `1` and `0`, respectively. Moreover, you can perform math operations of the `1`s and `0`s where they are casted as a numeric type. Indeed:

In [3]:
%%runcpp
#include <iostream>
using namespace std;

int main() {
    auto print_type = [](auto expr) {
        cout << "(" << typeid(expr).name() << ") " << expr << endl;
    };

    print_type(true);
    print_type(false);
    cout << (true || false) << endl;
    cout << (true && false) << endl;
    print_type(true + true);
    print_type(true * 1.);
    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

(b) 1
(b) 0
1
0
(i) 2
(d) 1


Boolean data are also used as results for things like comparison (`<`, `==`, etc.). As usual, relational and logical operators can be combined to form complex logical conditions. The following function demonstrates relational operators:

In [4]:
%%runcpp
#include <iostream>
using namespace std;

int main(){
    cout << (5 == 10) << endl;
    cout << (10 > 5) << endl;
    cout << ((5 >= 1) && (5 <= 10)) << endl;
    cout << ((5 >= 1) || (1 / 0)) << endl;
    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

0
1
1
1


**Remark.** Observe **short circuit** with OR works as `1/0` was not evaluated.

One gotcha with booleans is assignment. If you assign a non-null (i.e. nonzero), then it becomes evaluated as `1`. That is, `false = 0` and `!false = true`. To see this:

In [5]:
%%runcpp
#include <iostream>
using namespace std;

int main(){
    bool bool_var = -1;
    cout << bool_var << endl;
    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

1


Assigning -1 to a `bool` evaluates to 1! But this is intended behavior due to static typing.

## Character data

In Python strings can be created with single or double quotes. In C++ single quotes (') are used for the character (`char`) data type, and double quotes (") are used for the `string` data type.

Strings in C++ provides dynamic memory management of an array of chars. It supports built-in methods (e.g. `.size()`, `.substr()`, `.find()`), and features safety and flexibility over raw C-style strings (e.g. `char cstr[] = "hello";`). Note that `"a" == 'a'` results in an error in C++.

```c++
char a = 'h';
char cstr[] = "hello";      // C-style string (char array)
std::string str = "hello";  // C++ string object
```

But you can convert between them:

```c++
str.c_str();         // std::string → const char*
std::string(cstr);   // char* → std::string
```

## Pointers

In C++, an assignment like below consists of (1) allocating space in memory and (2) storing the value of a variable:

```c++
int var = 100;
```

To get the address of a variable in memory we use the **address-of** operator `&`:

In [6]:
%%runcpp
#include <iostream>
using namespace std;

int main(){
    int var = 100;
    cout << &var << " " << var << endl;
    var = 200;
    cout << &var << " " << var << endl;
    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

0x16d0f6d6c 100
0x16d0f6d6c 200


**Remark.** For basic types declared in a function scope, the address is fixed during their lifetime.

In Python, it is impossible to store a variable directly. Instead, a **reference** to the data object is used. In C++, variables store values directly, because they are faster to reference. References are slower, but they are sometimes useful. If in C++, we want to create an analogous reference to a memory location, we must use a special data type called a **pointer**.


### Pointer syntax

**Pointer init.** The syntax for initializing a pointer is similar to the usual assignment except we use `*` between the data type and the identifier:

```c++
int *ptr;
```

In [7]:
%%runcpp
#include <iostream>
using namespace std;

int main(){
    int *ptr;
    cout << ptr << endl;
    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

0x585b00018c66b140


### Assigning an address using `&`

One way to do this is to have a pointer refer to another variable by using the address-of operator `&`, which we know returns the address of a variable.

In [8]:
%%runcpp
#include <iostream>
using namespace std;

int main(){
    int var = 100;
    int *ptr = &var;
    cout << ptr << endl;     // address of var
    cout << *ptr << endl;    // dereference to get value

    // trying out reassignment
    var = 200;
    cout << &var << endl;
    cout << *ptr << endl;
    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

0x16d326d64
100
0x16d326d64
200


**Remark.** If we assign an address for a location that is outside of your **segment** (area in memory reserved for your program), the operating system will jump in with a message about a "**segmentation fault**" (aka **segfault**). (Although such an error message looks bad, a seg fault is in fact a helpful error because unlike the elusive logical errors, the reason is fairly localized.)

### Null pointer

Like `None` in Python, the null pointer (`nullptr`) in C++ points to nothing. The null pointer is often used in conditions AND / OR in logical operations. The ff. example shows how the null pointer works. We 

The following example demonstrates how the null pointer works. The variable `ptrx` initially has the address of `x` when it is declared. On the first iteration of the loop, it is assigned the value of nullptr, which evaluates to a false value; thereby ending the loop:



In [9]:
%%runcpp --exitcode=true
#include <iostream>
using namespace std;

int main() {
    int x = 12345;
    int *ptrx = &x;

    while (ptrx) {
        cout << "Pointer ptrx points to " << ptrx << endl;
        ptrx = nullptr;
    }

    cout << "Pointer ptrx points to nothing: " << ptrx << endl;
    cout << *ptrx << "segfault?" << endl;

    return 0;
}

g++-15 ./code/tmp.cpp -o ./code/tmp
./code/tmp

Pointer ptrx points to 0x16bb82d64
Pointer ptrx points to nothing: 0
-11


**NOTE:** Dereferencing the null pointer results in undefined behavior. Hence, the program crashed before it can print the last string ([-11 = segfault](https://docs.python.org/3/library/subprocess.html#subprocess.CompletedProcess.returncode)).